<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Graph_Neural_Networks_(GNNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch_geometric

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

# Ensure reproducibility
torch.manual_seed(42)

# Example graph data (nodes and edges)
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)  # Edges in source/target format
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)  # Node features

# Create graph data object
data = Data(x=x, edge_index=edge_index)

# Visualize the graph structure
def visualize_graph(data):
    G = to_networkx(data, to_undirected=True)  # Convert to NetworkX graph
    nx.draw(G, with_labels=True, node_color="lightblue", node_size=500, font_size=10)
    plt.title("Graph Visualization")
    plt.show()

visualize_graph(data)

# Define a GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(1, 16)  # From 1 input feature to 16 hidden features
        self.conv2 = GCNConv(16, 1)  # From 16 hidden features back to 1 output feature

    def forward(self, data):
        x, edge_index = data.x, data.edge_index  # Get node features and edges
        x = self.conv1(x, edge_index)  # First GCN layer
        x = F.relu(x)  # Apply ReLU activation
        x = self.conv2(x, edge_index)  # Second GCN layer
        return x

# Initialize the GCN model
model = GCN()

# Forward pass
output = model(data)
print("GCN output:")
print(output)